In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.sql import text

In [2]:
#Reading in the csv files
csv_file1 = "Resources/netflix_titles.csv"
csv_file2 = "Resources/movie_metadata.csv"
netflix_movies = pd.read_csv(csv_file1)
other_movies = pd.read_csv(csv_file2)
#Showing the first few rows of each file
netflix_movies.head()
#other_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
#extract relevant columns from netflix_movies
new_netflix_movies = netflix_movies[['title', 'type', 'country', 'director', 'release_year', 'rating']]
new_netflix_movies.head()

,title,type,country,director,release_year,rating
0,Norm of the North: King Sized Adventure,Movie,"United States, India, South Korea, China","Richard Finn, Tim Maltby",2019,TV-PG
1,Jandino: Whatever it Takes,Movie,United Kingdom,NaN,2016,TV-MA
2,Transformers Prime,TV Show,United States,NaN,2013,TV-Y7-FV
3,Transformers: Robots in Disguise,TV Show,United States,NaN,2016,TV-Y7
4,#realityhigh,Movie,United States,Fernando Lebrija,2017,TV-14


In [4]:
#drop TV shows from the column 'type' because we are interested in Movies only
cleaned_netflix = new_netflix_movies[new_netflix_movies.type != 'TV Show']
cleaned_netflix.head()

,title,type,country,director,release_year,rating
0,Norm of the North: King Sized Adventure,Movie,"United States, India, South Korea, China","Richard Finn, Tim Maltby",2019,TV-PG
1,Jandino: Whatever it Takes,Movie,United Kingdom,NaN,2016,TV-MA
4,#realityhigh,Movie,United States,Fernando Lebrija,2017,TV-14
6,Automata,Movie,"Bulgaria, United States, Spain, Canada",Gabe Ibáñez,2014,R
7,Fabrizio Copano: Solo pienso en mi,Movie,Chile,"Rodrigo Toro, Francisco Schultz",2017,TV-MA


In [5]:
#extract relevant columns from other_movies
new_other_movies = other_movies[['movie_title', 'genres', 'director_name', 'title_year', 'country', 'budget', 'gross', 'imdb_score', 'content_rating']]
new_other_movies.head()

,movie_title,genres,director_name,title_year,country,budget,gross,imdb_score,content_rating
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,2009.0,USA,237000000.0,760505847.0,7.9,PG-13
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,2007.0,USA,300000000.0,309404152.0,7.1,PG-13
2,Spectre,Action|Adventure|Thriller,Sam Mendes,2015.0,UK,245000000.0,200074175.0,6.8,PG-13
3,The Dark Knight Rises,Action|Thriller,Christopher Nolan,2012.0,USA,250000000.0,448130642.0,8.5,PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,NaN,NaN,NaN,NaN,7.1,NaN


In [7]:
#rename columns to match for simpler analysis 
new_other_movies.rename(columns={'movie_title': 'title', 'director_name': 'director', 'title_year': 'release_year', 'content_rating':'rating'},inplace=True)
cleaned_other_movies = new_other_movies
cleaned_other_movies.head()

,title,genres,director,release_year,country,budget,gross,imdb_score,rating
0,Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,2009.0,USA,237000000.0,760505847.0,7.9,PG-13
1,Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,2007.0,USA,300000000.0,309404152.0,7.1,PG-13
2,Spectre,Action|Adventure|Thriller,Sam Mendes,2015.0,UK,245000000.0,200074175.0,6.8,PG-13
3,The Dark Knight Rises,Action|Thriller,Christopher Nolan,2012.0,USA,250000000.0,448130642.0,8.5,PG-13
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,NaN,NaN,NaN,NaN,7.1,NaN


In [8]:
#reset index as title 
cleaned_netflix.set_index(['title'])

,type,country,director,release_year,rating
title,,,,,
Norm of the North: King Sized Adventure,Movie,"United States, India, South Korea, China","Richard Finn, Tim Maltby",2019,TV-PG
Jandino: Whatever it Takes,Movie,United Kingdom,NaN,2016,TV-MA
#realityhigh,Movie,United States,Fernando Lebrija,2017,TV-14
Automata,Movie,"Bulgaria, United States, Spain, Canada",Gabe Ibáñez,2014,R
Fabrizio Copano: Solo pienso en mi,Movie,Chile,"Rodrigo Toro, Francisco Schultz",2017,TV-MA
...,...,...,...,...,...
Frank and Cindy,Movie,United States,G.J. Echternkamp,2007,TV-MA
Frank and Cindy,Movie,United States,G.J. Echternkamp,2015,R
Iverson,Movie,United States,Zatella Beatty,2014,NR


In [9]:
#reset index as title 
cleaned_other_movies.set_index(['title'])

,genres,director,release_year,country,budget,gross,imdb_score,rating
title,,,,,,,,
Avatar,Action|Adventure|Fantasy|Sci-Fi,James Cameron,2009.0,USA,237000000.0,760505847.0,7.9,PG-13
Pirates of the Caribbean: At World's End,Action|Adventure|Fantasy,Gore Verbinski,2007.0,USA,300000000.0,309404152.0,7.1,PG-13
Spectre,Action|Adventure|Thriller,Sam Mendes,2015.0,UK,245000000.0,200074175.0,6.8,PG-13
The Dark Knight Rises,Action|Thriller,Christopher Nolan,2012.0,USA,250000000.0,448130642.0,8.5,PG-13
Star Wars: Episode VII - The Force Awakens,Documentary,Doug Walker,NaN,NaN,NaN,NaN,7.1,NaN
...,...,...,...,...,...,...,...,...
Signed Sealed Delivered,Comedy|Drama,Scott Smith,2013.0,Canada,NaN,NaN,7.7,NaN
The Following,Crime|Drama|Mystery|Thriller,NaN,NaN,USA,NaN,NaN,7.5,TV-14
A Plague So Pleasant,Drama|Horror|Thriller,Benjamin Roberds,2013.0,USA,1400.0,NaN,6.3,NaN


In [10]:
#print CSV
cleaned_netflix.to_csv(r'cleaned_netflix.csv')

In [11]:
#print CSV
cleaned_other_movies.to_csv(r'cleaned_other_movies.csv')

In [12]:
#create connection with PostgreSQL
rds_connection_string = "postgres:password@localhost:5432/ETLproject"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
#displayed table names from our database
engine.table_names()

['movies', 'netflix']

In [14]:
pd.read_sql_query('select * from netflix', con=engine).head()

,index,title,type,country,director,release_year,rating
0,1,Jandino: Whatever it Takes,Movie,United Kingdom,None,2016,TV-MA
1,4,#realityhigh,Movie,United States,Fernando Lebrija,2017,TV-14
2,6,Automata,Movie,"Bulgaria, United States, Spain, Canada",Gabe Ibáñez,2014,R
3,7,Fabrizio Copano: Solo pienso en mi,Movie,Chile,"Rodrigo Toro, Francisco Schultz",2017,TV-MA
4,9,Good People,Movie,"United States, United Kingdom, Denmark, Sweden",Henrik Ruben Genz,2014,R


In [15]:
#remove spaces and special characters to join on matching titles 
sql = text("""SELECT * FROM netflix,movies \
where REGEXP_REPLACE(netflix.title, '[^0-9a-zA-Z.,]+$', '') = REGEXP_REPLACE(movies.title, '[^0-9a-zA-Z.,]+$', '') \
order by movies.title""")

In [16]:
final_csv = pd.read_sql_query(sql, con=engine)

In [17]:
final_csv.to_csv(r'final csv.csv')

In [18]:
#drop duplicate 'rating' column 
final_csv = final_csv.iloc[:, [j for j, c in enumerate(final_csv.columns) if j != 16]]
final_csv.head()


,index,title,type,country,director,release_year,rating,index,title,genres,director,release_year,country,budget,gross,imdb_score
0,2695,"10,000 B.C.",Movie,"United States, South Africa",Roland Emmerich,2008,PG-13,279,"10,000 B.C.",Comedy,Christopher Barnard,NaN,None,NaN,NaN,7.2
1,1343,16 Blocks,Movie,"United States, Germany",Richard Donner,2006,PG-13,1115,16 Blocks,Action|Crime|Drama|Thriller,Richard Donner,2006.0,Germany,52000000.0,36883539.0,6.6
2,5390,21 & Over,Movie,United States,"Jon Lucas, Scott Moore",2013,R,2875,21 & Over,Comedy,Jon Lucas,2013.0,USA,13000000.0,25675765.0,5.9
3,3480,21,Movie,United States,Robert Luketic,2008,PG-13,1412,21,Crime|Drama|Thriller,Robert Luketic,2008.0,USA,35000000.0,81159365.0,6.8
4,2696,50/50,Movie,United States,Jonathan Levine,2011,R,3368,50/50,Comedy|Drama|Romance,Jonathan Levine,2011.0,USA,8000000.0,34963967.0,7.7


In [19]:
#analyze 'rating' column 
final_csv['rating'].value_counts()

R        201
PG-13    153
PG        72
TV-MA     25
TV-14     14
G         10
TV-PG      8
NR         6
UR         2
TV-Y7      1
Name: rating, dtype: int64

In [22]:
#drop duplicate 'release_year' column
final_csv = final_csv.iloc[:, [j for j, c in enumerate(final_csv.columns) if j != 11]]
final_csv.head()

,index,title,type,country,director,release_year,rating,index,title,genres,director,country,budget,gross,imdb_score
0,2695,"10,000 B.C.",Movie,"United States, South Africa",Roland Emmerich,2008,PG-13,279,"10,000 B.C.",Comedy,Christopher Barnard,None,NaN,NaN,7.2
1,1343,16 Blocks,Movie,"United States, Germany",Richard Donner,2006,PG-13,1115,16 Blocks,Action|Crime|Drama|Thriller,Richard Donner,Germany,52000000.0,36883539.0,6.6
2,5390,21 & Over,Movie,United States,"Jon Lucas, Scott Moore",2013,R,2875,21 & Over,Comedy,Jon Lucas,USA,13000000.0,25675765.0,5.9
3,3480,21,Movie,United States,Robert Luketic,2008,PG-13,1412,21,Crime|Drama|Thriller,Robert Luketic,USA,35000000.0,81159365.0,6.8
4,2696,50/50,Movie,United States,Jonathan Levine,2011,R,3368,50/50,Comedy|Drama|Romance,Jonathan Levine,USA,8000000.0,34963967.0,7.7


In [35]:
(final_csv['release_year'].value_counts(ascending = False))

2009    37
2008    32
2010    31
2006    25
2007    25
2014    24
2013    24
2015    21
1997    20
2011    20
2002    19
2000    17
2004    17
2003    17
2005    16
2001    14
2012    14
2016    12
2019    11
1998    10
2017    10
1999     9
2018     9
1996     7
1976     6
1994     6
1993     4
1973     3
1995     3
1992     3
1988     2
1962     2
1984     2
1981     2
1968     2
1991     2
1974     2
1990     1
1971     1
1960     1
1965     1
1969     1
1975     1
1989     1
1978     1
1980     1
1986     1
1987     1
1958     1
Name: release_year, dtype: int64

In [36]:
#drop duplicate 'director' column
final_csv = final_csv.iloc[:, [j for j, c in enumerate(final_csv.columns) if j != 10]]
final_csv.head()

,index,title,type,country,director,release_year,rating,index,title,genres,country,budget,gross,imdb_score
0,2695,"10,000 B.C.",Movie,"United States, South Africa",Roland Emmerich,2008,PG-13,279,"10,000 B.C.",Comedy,None,NaN,NaN,7.2
1,1343,16 Blocks,Movie,"United States, Germany",Richard Donner,2006,PG-13,1115,16 Blocks,Action|Crime|Drama|Thriller,Germany,52000000.0,36883539.0,6.6
2,5390,21 & Over,Movie,United States,"Jon Lucas, Scott Moore",2013,R,2875,21 & Over,Comedy,USA,13000000.0,25675765.0,5.9
3,3480,21,Movie,United States,Robert Luketic,2008,PG-13,1412,21,Crime|Drama|Thriller,USA,35000000.0,81159365.0,6.8
4,2696,50/50,Movie,United States,Jonathan Levine,2011,R,3368,50/50,Comedy|Drama|Romance,USA,8000000.0,34963967.0,7.7


In [37]:
(final_csv['director'].value_counts(ascending = False))

Steven Spielberg     8
Quentin Tarantino    6
Sam Raimi            5
Martin Scorsese      5
Kevin Smith          5
                    ..
Philip G. Atwell     1
Gus Van Sant         1
Vrinda Samartha      1
Andrew Fleming       1
Andrew Douglas       1
Name: director, Length: 378, dtype: int64

In [39]:
#analyze the 'imdb_score' column by creating bins for the different score values
imdb=final_csv['imdb_score'].value_counts(bins=5)
imdb

(6.1, 7.6]                   256
(4.6, 6.1]                   130
(7.6, 9.1]                    84
(3.1, 4.6]                    19
(1.5910000000000002, 3.1]      3
Name: imdb_score, dtype: int64